In [ ]:
!pip install nltk -q
!pip install gradio -q
!pip install sentence-transformers -q

import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
stop = stopwords.words('english')
print(len(stop))

     |████████████████████████████████| 11.6 MB 22.1 MB/s 
     |████████████████████████████████| 56 kB 5.0 MB/s 
     |████████████████████████████████| 84 kB 3.8 MB/s 
     |████████████████████████████████| 213 kB 98.0 MB/s 
     |████████████████████████████████| 55 kB 4.2 MB/s 
     |████████████████████████████████| 2.3 MB 83.8 MB/s 
     |████████████████████████████████| 272 kB 77.2 MB/s 
     |████████████████████████████████| 54 kB 3.8 MB/s 
     |████████████████████████████████| 106 kB 76.9 MB/s 
     |████████████████████████████████| 84 kB 4.5 MB/s 
     |████████████████████████████████| 64 kB 2.5 MB/s 
     |████████████████████████████████| 80 kB 11.0 MB/s 
     |████████████████████████████████| 68 kB 7.3 MB/s 
     |████████████████████████████████| 46 kB 4.6 MB/s 
     |████████████████████████████████| 856 kB 59.6 MB/s 
     |████████████████████████████████| 4.1 MB 60.6 MB/s 
     |████████████████████████████████| 593 kB 86.3 MB/s 
     |████████████████████████

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from sentence_transformers import SentenceTransformer

from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
df = pd.read_csv('titles.csv')
print(df.shape)
df.head()

(119453, 1)


,title
0,Solving QBF Instances with Nested SAT Solvers.
1,Multiscale Manifold Learning.
2,Online Transfer Learning for Differential Diag...
3,Logic Programming in Assumption-Based Argument...
4,Role-Aware Conformity Modeling and Analysis in...


In [10]:
import pickle

df2 = pd.DataFrame()
with open('original_titles.pkl', 'rb') as file:
  df2['title'] = pickle.load(file)

df2.head()

,title
0,Solving QBF Instances with Nested SAT Solvers.
1,Multiscale Manifold Learning.
2,Online Transfer Learning for Differential Diag...
3,Logic Programming in Assumption-Based Argument...
4,Role-Aware Conformity Modeling and Analysis in...


In [7]:
sample = df.sample(50)
sample = sample.reset_index(drop=True)
sample.head()

,title
0,EventSearch: a system for event discovery and ...
1,Better Inapproximability Results for MaxClique...
2,Classification rule learning for data linking.
3,Digitizer auditory graph: making graphs access...
4,BitCuts: Towards Fast Packet Classification fo...


In [12]:
# Cleaning

sample['text'] = sample['title'].str.lower()
sample['text'].apply(lambda item: " ".join(item for item in item.split() if item not in stop))
sample.head()

,title,text
0,EventSearch: a system for event discovery and ...,eventsearch: a system for event discovery and ...
1,Better Inapproximability Results for MaxClique...,better inapproximability results for maxclique...
2,Classification rule learning for data linking.,classification rule learning for data linking.
3,Digitizer auditory graph: making graphs access...,digitizer auditory graph: making graphs access...
4,BitCuts: Towards Fast Packet Classification fo...,bitcuts: towards fast packet classification fo...


In [14]:
model = SentenceTransformer('sentence-transformers/multi-qa-MiniLM-L6-cos-v1'
)

In [15]:
print(model)

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)


In [16]:
features = model.encode(sample['text'])
sample['features'] = list(features)
sample.head()

,title,text,features
0,EventSearch: a system for event discovery and ...,eventsearch: a system for event discovery and ...,"[-0.027753772, 0.0046088025, -0.050095476, -0...."
1,Better Inapproximability Results for MaxClique...,better inapproximability results for maxclique...,"[-0.02947729, -0.037161093, 0.041651044, -0.10..."
2,Classification rule learning for data linking.,classification rule learning for data linking.,"[-0.0626378, -0.017450066, -0.01587462, -0.012..."
3,Digitizer auditory graph: making graphs access...,digitizer auditory graph: making graphs access...,"[0.056223173, -0.04559535, 0.04206844, -0.0714..."
4,BitCuts: Towards Fast Packet Classification fo...,bitcuts: towards fast packet classification fo...,"[-0.083068684, -0.035300985, -0.024230598, -0...."


In [17]:
def search(query_text):
  query_feature = model.encode([query_text])
  sample['similarity'] = sample['features'].apply(lambda x: cosine_similarity([x], query_feature)[0][0])
  sorted = sample.sort_values(by='similarity', ascending=False)
  return sorted.head(10)

In [24]:
def results(query_text):
  query_feature = model.encode([query_text])
  sample['similarity'] = sample['features'].apply(lambda x: cosine_similarity([x], query_feature)[0][0])
  sorted = sample.sort_values(by='similarity', ascending=False)
  result = sorted['title'].head(5)
  return list(result)

In [20]:
search(str(input("Enter a query: \t")))

Enter a query: 	math


,title,text,features,similarity
39,"Demo: software defined radio: on a smartphone,...","demo: software defined radio: on a smartphone,...","[-0.024068294, 0.01936715, -0.04585923, -0.097...",0.142202
13,Quantitative assessment with using ticket driv...,quantitative assessment with using ticket driv...,"[0.103615016, -0.014896829, -0.073657714, 0.00...",0.142046
49,Envisioning ubiquitous computing.,envisioning ubiquitous computing.,"[-0.03082905, -0.045786113, -0.062799625, -0.0...",0.125291
37,Sublinear Distance Labeling.,sublinear distance labeling.,"[-0.02996075, 0.016225323, -0.055561647, -0.13...",0.108142
18,Overview of Data Exploration Techniques.,overview of data exploration techniques.,"[-0.014134068, 0.03571273, 0.0019148361, 0.035...",0.103121
34,Octant: A Comprehensive Framework for the Geol...,octant: a comprehensive framework for the geol...,"[0.0220426, 0.051018946, -0.008712557, -0.0250...",0.095972
43,Optimizing relevance and revenue in ad search:...,optimizing relevance and revenue in ad search:...,"[0.034534175, 0.038633704, -0.010609549, -0.08...",0.091313
28,A Polynomial Algorithm for Weighted Abstract F...,a polynomial algorithm for weighted abstract f...,"[-0.10667588, 0.044135816, -0.026798224, 0.077...",0.082720
15,Calculating latent demand in the long tail.,calculating latent demand in the long tail.,"[-0.031142192, 0.021725403, 0.009248563, -0.01...",0.080352
16,A characterization of online browsing behavior.,a characterization of online browsing behavior.,"[-0.0090928115, -0.0738521, 0.031461287, -0.01...",0.056431


In [28]:
import gradio as gr

u_interface = gr.Interface(fn=results, inputs="text", outputs="text")
u_interface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://983dc91551c30a2e.gradio.app

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


(<gradio.routes.App at 0x7f73516e6c90>,
 'http://127.0.0.1:7861/',
 'https://983dc91551c30a2e.gradio.app')